In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv(r"D:\Developer\Datasets\AI_club_inductions\training_data.csv")
ld_train_data = pd.read_csv(r"D:\Developer\Datasets\Loan_deafulters\train_LZV4RXX.csv")

test_data = pd.read_csv(r"D:\Developer\Datasets\AI_club_inductions\test_data.csv")
ld_test_data = pd.read_csv(r"D:\Developer\Datasets\Loan_deafulters\test_4zJg83n.csv")

In [ ]:
data_corr = train_data.corr()
loan_default_corr = data_corr['loan_default']
print(len(train_data))
print(len(test_data))

5620
1400


C:\Users\pprav\AppData\Local\Temp\ipykernel_15284\4147724482.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_corr = train_data.corr()


In [ ]:
net_train_data = pd.concat([train_data, ld_train_data], axis = 0)
net_test_data = test_data
total_train_data = pd.concat([net_train_data, net_test_data], axis = 0)


# total_train_data = pd.concat([train_data, test_data], axis = 0)
print(total_train_data)

      loan_id  age  education proof_submitted  loan_amount  asset_cost  \
0        1033   28        1.0          Aadhar     777600.0    961200.0   
1        6340   39        2.0          Aadhar     463848.0   1046532.0   
2        3887   27        2.0          Aadhar     654984.0    865380.0   
3        2654   29        2.0         VoterID     691716.0    867792.0   
4        6915   37        1.0          Aadhar     796584.0    961200.0   
...       ...  ...        ...             ...          ...         ...   
1395     3485   32        1.0          Aadhar     639636.0    879276.0   
1396     1861   27        1.0          Aadhar     589740.0    875316.0   
1397     4975   34        2.0          Aadhar     619836.0    964812.0   
1398      388   28        2.0          Aadhar     801228.0   1321200.0   
1399     4337   28        1.0          Aadhar     425268.0    797520.0   

      no_of_loans  no_of_curr_loans  last_delinq_none  loan_default  
0               0                 0      

In [ ]:
from sklearn.impute import SimpleImputer

features = ["loan_id", "age", "education","proof_submitted", "loan_amount", "asset_cost", "no_of_loans", "no_of_curr_loans", "last_delinq_none"]
new_total_train_data = total_train_data[features]

#X.head()

imputer = SimpleImputer(strategy='constant', fill_value=0)

imputed_total_train = pd.DataFrame(imputer.fit_transform(new_total_train_data))

imputed_total_train.columns = features
#imputed_X_val.columns = features

imputed_total_train = imputed_total_train.apply(lambda col: col.astype(float) if col.name != 'proof_submitted' else col)

imputed_total_train.head()

s = (imputed_total_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['proof_submitted']


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaled_down = ['loan_amount', 'asset_cost']
scaler = MinMaxScaler()
scaler.fit(imputed_total_train[scaled_down])
scaled_total_train = (scaler.transform(imputed_total_train[scaled_down]))

imputed_total_train[scaled_down] = scaled_total_train

#imputed_X_train.columns = imputed_X_train.columns.astype(str)
#imputed_X_val.columns = imputed_X_val.columns.astype(str)

In [ ]:
imputed_total_train.head()

,loan_id,age,education,proof_submitted,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none
0,1033.0,28.0,1.0,Aadhar,0.377888,0.250712,0.0,0.0,0.0
1,6340.0,39.0,2.0,Aadhar,0.183433,0.294566,5.0,3.0,0.0
2,3887.0,27.0,2.0,Aadhar,0.301894,0.201469,2.0,0.0,0.0
3,2654.0,29.0,2.0,VoterID,0.324659,0.202709,0.0,0.0,0.0
4,6915.0,37.0,1.0,Aadhar,0.389653,0.250712,0.0,0.0,0.0


In [ ]:
for col in imputed_total_train.select_dtypes(include = 'object').columns:
    print(col, imputed_total_train[col].nunique())

proof_submitted 7


In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

one_hot_cols_train = pd.DataFrame(one_hot_encoder.fit_transform(imputed_total_train[object_cols]), columns=one_hot_encoder.get_feature_names_out(['proof_submitted']))
one_hot_cols_train.columns = one_hot_cols_train.columns.astype(str)

# one_hot_cols_val = pd.DataFrame(one_hot_encoder.transform(imputed_X_val[object_cols]), columns=one_hot_encoder.get_feature_names_out(['proof_submitted']))
# one_hot_cols_val.columns = one_hot_cols_val.columns.astype(str)

one_hot_cols_train.head()
                                  
new_total_train = imputed_total_train.drop(object_cols, axis=1)
# new_X_val = imputed_X_val.drop(object_cols, axis=1)

imputed_one_hot_total_train = pd.concat([new_total_train, one_hot_cols_train], axis=1)
# imputed_one_hot_X_val = pd.concat([new_X_val, one_hot_cols_val], axis=1)

print(imputed_one_hot_total_train)

       loan_id   age  education  loan_amount  asset_cost  no_of_loans  \
0       1033.0  28.0        1.0     0.377888    0.250712          0.0   
1       6340.0  39.0        2.0     0.183433    0.294566          5.0   
2       3887.0  27.0        2.0     0.301894    0.201469          2.0   
3       2654.0  29.0        2.0     0.324659    0.202709          0.0   
4       6915.0  37.0        1.0     0.389653    0.250712          0.0   
...        ...   ...        ...          ...         ...          ...   
14015   3485.0  32.0        1.0     0.292381    0.208610          0.0   
14016   1861.0  27.0        1.0     0.261457    0.206575          5.0   
14017   4975.0  34.0        2.0     0.280110    0.252569          0.0   
14018    388.0  28.0        2.0     0.392531    0.435722          0.0   
14019   4337.0  28.0        1.0     0.159522    0.166595          0.0   

       no_of_curr_loans  last_delinq_none  proof_submitted_Aadhar  \
0                   0.0               0.0             

c:\Users\pprav\miniconda3\envs\env\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
split_index = len(net_train_data)
test_split_index = len(test_data)
print(split_index)
final_train_data = imputed_one_hot_total_train.iloc[:split_index]
final_test_data = imputed_one_hot_total_train.iloc[split_index:]
# bot_test_data = final_test_data.iloc[:test_split_index]
print(final_test_data)

# split_index = len(train_data)
# final_train_data = imputed_one_hot_total_train.iloc[:split_index]
# final_test_data = imputed_one_hot_total_train.iloc[split_index:]
# print(final_test_data)

12620
       loan_id   age  education  loan_amount  asset_cost  no_of_loans  \
12620   6501.0  27.0        1.0     0.354929    0.194593          5.0   
12621   2945.0  44.0        2.0     0.303909    0.176314          3.0   
12622   2025.0  35.0        1.0     0.235724    0.151800          0.0   
12623    264.0  43.0        2.0     0.181462    0.167026          0.0   
12624   4351.0  40.0        2.0     0.327411    0.163758          2.0   
...        ...   ...        ...          ...         ...          ...   
14015   3485.0  32.0        1.0     0.292381    0.208610          0.0   
14016   1861.0  27.0        1.0     0.261457    0.206575          5.0   
14017   4975.0  34.0        2.0     0.280110    0.252569          0.0   
14018    388.0  28.0        2.0     0.392531    0.435722          0.0   
14019   4337.0  28.0        1.0     0.159522    0.166595          0.0   

       no_of_curr_loans  last_delinq_none  proof_submitted_Aadhar  \
12620               0.0               0.0       

In [ ]:
from sklearn.model_selection import train_test_split

y = net_train_data.loan_default
# y = train_data.loan_default
X = final_train_data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.head()

,loan_id,age,education,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none,proof_submitted_Aadhar,proof_submitted_Driving,proof_submitted_Other,proof_submitted_PAN,proof_submitted_PANDriving,proof_submitted_Passport,proof_submitted_VoterID
5359,4738.0,37.0,2.0,0.287919,0.181439,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8553,2934.0,48.0,2.0,0.332959,0.250096,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7333,1714.0,42.0,2.0,0.270069,0.180304,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11953,6334.0,50.0,2.0,0.197578,0.086227,20.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6220,601.0,35.0,2.0,0.417476,0.286629,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=6, min_samples_split = 2)
model = XGBClassifier(random_state = 0, n_estimators = 1000, learning_rate = 0.17, max_depth = 6)
# model = XGBClassifier()
# model = LogisticRegression(C = 1.0, penalty = 'l2', solver = 'newton-cg')
# model.get_params()
#For Logistic Regressor
# solvers = ['newton-cg', 'lbfgs', 'liblinear']
# penalty = ['l2']
# c_values = [100, 10, 1.0, 0.1, 0.01]
# # define grid search
# grid = dict(solver=solvers,penalty=penalty,C=c_values)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)

# For XGBoost
# maxidepth = 1, 2, 3, 4, 5, 6
# learn_rate = 0.01, 0.05, 0.1, 0.17, 0.20
# random_state = 0, 1, 2, 3, 4
# n_estim = 100, 220, 500, 1000,
# min_child_weight = 1, 2, 3, 4
# subsample = 0.1, 0.25, 0.5, 0.75, 1
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# grid = dict(max_depth = maxidepth, learning_rate = learn_rate, random_state = random_state, n_estimators = n_estim, subsample = subsample, min_child_weight = min_child_weight)
# print(grid)
# grid_search = GridSearchCV(
#     estimator=model,
#     param_grid=grid,
#     scoring='roc_auc',
#     n_jobs=1,
#     cv=cv,
#     verbose=1
# )

# grid_search.fit(X_train, y_train)
# print(grid_search.best_params_)

{'max_depth': (1, 2, 3, 4, 5, 6), 'learning_rate': (0.01, 0.05, 0.1, 0.17, 0.2), 'random_state': (0, 1, 2, 3, 4), 'n_estimators': (100, 220, 500, 1000), 'subsample': (0.1, 0.25, 0.5, 0.75, 1), 'min_child_weight': (1, 2, 3, 4)}
Fitting 30 folds for each of 12000 candidates, totalling 360000 fits


KeyboardInterrupt: 

In [ ]:
model.fit(X_train, y_train)
y_predict = model.predict(X_val)
model_accuracy = accuracy_score(y_val, y_predict)

#print(final_test_data)

prediction = model.predict(final_test_data)
print(model_accuracy)

0.7555467511885895
